## Cooking with ClarityNLP - Session #6

Today we'll be discussing setup and configuration of ClarityNLP. We'll discuss system requirements, installation steps, report type mapping and data ingestion.

### System Requirements:

1. Install [Docker for Mac](https://www.docker.com/docker-mac) or [Docker for Windows](https://www.docker.com/docker-windows)

2. These are the recommended Docker settings for ClarityNLP. In Docker, they can be updated via ```Docker > Preferences > Advanced:```

    * Memory: $>8$ GB
    * Disk: $>256$ GB recommended, but can run on much less (depends on data needs)

<div> <img src="./assets/docker_memory_config.png" alt="Adjusting Docker Memory Allocation" style="width: 300px;"/> </div>
 

### Installation via Git:

1. [Install git](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git) if you do not already have it installed on your machine. Alternatively, you can also [download](https://github.com/ClarityNLP/ClarityNLP/archive/master.zip) the ClarityNLP repository as a zip file. <br/><br/>

2. If you choose to clone the repository, in a terminal window, run:<br/><br/>
    ```git clone https://github.com/ClarityNLP/ClarityNLP``` <br/><br/>
    
3. Switch to the ClarityNLP directory <br/><br/>
    ```cd ClarityNLP``` <br/><br/>

4. Initialize submodules <br/><br/>
    ```git submodule update --init --recursive``` <br/><br/>

5. Add .env file, use ```.env.example``` as a start: <br/><br/>
    ```touch .env``` <br/><br/>
    ```cat .env.example >> .env```

### Post-Installlation via Docker:

1. Build images and run containers. This will take 30-60 minutes the first time. <br><br>
```sh run_claritynlp.sh``` <br><br>

    Note that while this script is running, you will observe many system initializations inside the terminal window. These correspond to ClarityNLP's dependencies, and will look something like this:
    
    <div> <img src="./assets/docker_compose_in_progress.png" alt="Docker Compose In-Progress (Mac)" style="width: 800px"/> </div><br><br>

2. Alternatively, you can call Docker directly: <br><br>
```docker-compose up --build``` <br><br>

3. To verify that containers are running, open a new terminal window and run:<br><br>
``` docker ps``` 

    The Luigi container will monitor for active tasks. It is expected to see the following output throughout the logs.

    ```LUIGI_SCHEDULER   | 2018-10-16 19:46:19,149 luigi.scheduler INFO     Starting pruning of task graph
    LUIGI_SCHEDULER   | 2018-10-16 19:46:19,149 luigi.scheduler INFO     Done pruning task graph``` <br><br>

4. To shut down ClarityNLP:<br><br>
    Hit ```CMD+C (Mac)``` or ```Ctrl+C (Windows)``` from the terminal window where ClarityNLP was started. 

    Alternatively, you can run the following shell script to forcefully kill the containers: 

    ```sh stop_all_docker_containers.sh``` 

### Trouble-shooting common roadblocks:

1. You may experience networking issues if you attempt to stand up ClarityNLP locally using Docker while on a VPN and/or behind a firewall. This is because Docker requires specific ports to be open/accessible, and VPN/firewall settings may interfere with these requirements. <br><br>

2. If your data is hosted in an air-gapped environment, you will need to set up ClarityNLP first (e.g., while connected to the Internet), and then move the VM to the air-gapped environment as a second step. <br><br>

3. If you experience port-related issues on a specific subset of port(s), check to make sure that you do not have any services running that conflict: <br><br>
<div><img src="./assets/docker_ports.png" alt="Docker Port Usage" style="width: 1100px"/> </div><br><br>

4. If you wish to use a hosted version of one of ClarityNLP's services (such as MongoDB, Solr, Postgres), you can comment in your `.env` file and modify your `docker-compose.yml` to setup those services. You will need to manually setup Postgres and Solr with the corresponding schemas found in the ClarityNLP `utilities/` directory.

### ClarityNLP Links:

ClarityNLP's user-facing services can be accessed via the links below: 

* ClarityNLP API <http://localhost:5000>
* ClarityNLP Solr <http://localhost:8983>
* ClarityNLP Luigi <http://localhost:8082>
* Report Type Mapper Docs <http://localhost:3000/>
* Report Type Mapper Client <http://localhost:8000>
* Ingest Client <http://localhost:8500/>_
* Results Client <http://localhost:8201/>

* ClarityNLP Postgres <jdbc:postgresql://localhost:5433/clarity>
* ClaritNLP Mongo <http://localhost:27017>
* Ingest Mongo <http://localhost:27020>

### Report-type mapping: